In [45]:
import numpy as np
import torch
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def fill_inf_with_max_min(arr: np.ndarray, axis: int = 0, backend: str = "numpy", device: str = "cpu") -> np.ndarray:
    """Replace inf/-inf with max/min non-inf values along axis, in-place"""
    if backend == "torch":
        arr_torch = torch.from_numpy(arr).double().to(device) if isinstance(arr, np.ndarray) else arr
        mask_inf_pos = torch.isinf(arr_torch) & (arr_torch > 0)
        mask_inf_neg = torch.isinf(arr_torch) & (arr_torch < 0)
        mask_nan = torch.isnan(arr_torch)
        arr_torch[mask_inf_pos | mask_inf_neg] = float('nan')  # Replace inf with NaN
        mean_val = torch.nanmean(arr_torch, dim=axis, keepdim=True)
        mean_val_broadcast = torch.broadcast_to(mean_val, arr_torch.shape)
        mask_invalid = mask_nan | mask_inf_neg | mask_inf_pos
        arr_torch[mask_invalid] = mean_val_broadcast[mask_invalid] 
        if mask_inf_pos.any() or mask_inf_neg.any():
            
            valid = arr_torch[~(mask_nan|mask_inf_neg|mask_inf_pos)]
            if valid.numel() > 0:
                logger.info(f"Valid elements shape: {valid}")
                max_val = torch.max(arr_torch, dim=axis, keepdim=True).values
                min_val = torch.min(arr_torch, dim=axis, keepdim=True).values
                max_val_broadcast = torch.broadcast_to(max_val, arr_torch.shape)
                min_val_broadcast = torch.broadcast_to(min_val, arr_torch.shape)
                logger.info(f"Valid  {valid}")
                logger.info(f"torch Max val: {max_val}, Min val: {min_val}")
                arr_torch[mask_inf_pos] = max_val_broadcast[mask_inf_pos]
                arr_torch[mask_inf_neg] = min_val_broadcast[mask_inf_neg]
                arr_torch[mask_nan] = float('nan')  # Ensure NaN remains NaN
            else:
                arr_torch[mask_inf_pos] = float('nan')
                arr_torch[mask_inf_neg] = float('nan')
        return arr_torch.cpu().numpy() if isinstance(arr, np.ndarray) else arr_torch
    else:
        mask_inf_pos = np.isinf(arr) & (arr > 0)
        mask_inf_neg = np.isinf(arr) & (arr < 0)
        arr[np.isinf(arr)] = np.nan
        if mask_inf_pos.any() or mask_inf_neg.any():
            valid = arr[~np.isnan(arr)]
            if valid.size > 0:
                max_val = np.nanmax(arr, axis=axis, keepdims=True)
                min_val = np.nanmin(arr, axis=axis, keepdims=True)
                logger.info(f"numpy Max value: {max_val}, Min value: {min_val}")
                max_val_broadcast = np.broadcast_to(max_val, arr.shape)
                min_val_broadcast = np.broadcast_to(min_val, arr.shape)
                arr[mask_inf_pos] = max_val_broadcast[mask_inf_pos]
                arr[mask_inf_neg] = min_val_broadcast[mask_inf_neg]
            else:
                arr[mask_inf_pos] = float('nan')
                arr[mask_inf_neg] = float('nan')
        return arr

# 测试用例
if __name__ == "__main__":
    # 测试 1：正常值 + inf
    arr1 = np.array([[1, 2], [float('inf'), 3], [-float('inf'), 4], [5, 6]], dtype=float)
    logger.info(f"Test 1: Normal values + inf {arr1}")
    numpy_result1 = fill_inf_with_max_min(arr1.copy(), axis=0, backend="numpy")
    torch_result1 = fill_inf_with_max_min(arr1.copy(), axis=0, backend="torch", device="cpu")
    print("NumPy Result 1:\n", numpy_result1)
    print("PyTorch Result 1:\n", torch_result1)
    print("Identical:", np.array_equal(numpy_result1, torch_result1, equal_nan=True))
    print("Close:", np.allclose(numpy_result1, torch_result1, rtol=1e-5, equal_nan=True))

    # 测试 2：含 NaN
    arr2 = np.array([[1, np.nan], [float('inf'), 3], [-float('inf'), 4], [5, 6]], dtype=float)
    logger.info("Test 2: Values + inf + NaN")
    numpy_result2 = fill_inf_with_max_min(arr2.copy(), axis=0, backend="numpy")
    torch_result2 = fill_inf_with_max_min(arr2.copy(), axis=0, backend="torch", device="cpu")
    print("NumPy Result 2:\n", numpy_result2)
    print("PyTorch Result 2:\n", torch_result2)
    print("Identical:", np.array_equal(numpy_result2, torch_result2, equal_nan=True))
    print("Close:", np.allclose(numpy_result2, torch_result2, rtol=1e-5, equal_nan=True))

    # 测试 3：全 inf 或 NaN
    arr3 = np.array([[float('inf'), float('inf')], [-float('inf'), np.nan]], dtype=float)
    logger.info("Test 3: All inf or NaN")
    numpy_result3 = fill_inf_with_max_min(arr3.copy(), axis=0, backend="numpy")
    torch_result3 = fill_inf_with_max_min(arr3.copy(), axis=0, backend="torch", device="cpu")
    print("NumPy Result 3:\n", numpy_result3)
    print("PyTorch Result 3:\n", torch_result3)
    print("Identical:", np.array_equal(numpy_result3, torch_result3, equal_nan=True))

INFO:__main__:Test 1: Normal values + inf [[  1.   2.]
 [ inf   3.]
 [-inf   4.]
 [  5.   6.]]
INFO:__main__:numpy Max value: [[5. 6.]], Min value: [[1. 2.]]
INFO:__main__:Valid elements shape: tensor([1., 2., 3., 4., 5., 6.], dtype=torch.float64)
INFO:__main__:Valid  tensor([1., 2., 3., 4., 5., 6.], dtype=torch.float64)
INFO:__main__:torch Max val: tensor([[5., 6.]], dtype=torch.float64), Min val: tensor([[1., 2.]], dtype=torch.float64)
INFO:__main__:Test 2: Values + inf + NaN
INFO:__main__:numpy Max value: [[5. 6.]], Min value: [[1. 3.]]
INFO:__main__:Valid elements shape: tensor([1., 3., 4., 5., 6.], dtype=torch.float64)
INFO:__main__:Valid  tensor([1., 3., 4., 5., 6.], dtype=torch.float64)
INFO:__main__:torch Max val: tensor([[5., 6.]], dtype=torch.float64), Min val: tensor([[1., 3.]], dtype=torch.float64)
INFO:__main__:Test 3: All inf or NaN


NumPy Result 1:
 [[1. 2.]
 [5. 3.]
 [1. 4.]
 [5. 6.]]
PyTorch Result 1:
 [[1. 2.]
 [5. 3.]
 [1. 4.]
 [5. 6.]]
Identical: True
Close: True
NumPy Result 2:
 [[ 1. nan]
 [ 5.  3.]
 [ 1.  4.]
 [ 5.  6.]]
PyTorch Result 2:
 [[ 1. nan]
 [ 5.  3.]
 [ 1.  4.]
 [ 5.  6.]]
Identical: True
Close: True
NumPy Result 3:
 [[nan nan]
 [nan nan]]
PyTorch Result 3:
 [[nan nan]
 [nan nan]]
Identical: True


In [ ]:
import numpy as np
import torch

def rolling_kurt(array, window, backend="numpy", device="cpu"):
    """
    Rolling unbiased excess kurtosis, matching pandas behavior.
    NumPy and PyTorch versions provided.
    """
    T, F = array.shape
    out = np.full((T, F), np.nan, dtype=float)

    # 1. NumPy backend
    if backend == "numpy":
        sw = np.lib.stride_tricks.sliding_window_view(array, window, axis=0)
        m = np.nanmean(sw, axis=2, keepdims=True)
        dev = sw - m
        m2 = np.nanmean(dev**2, axis=2)
        m4 = np.nanmean(dev**4, axis=2)
        n = np.sum(~np.isnan(sw), axis=2).astype(float)

        # Only compute for windows with at least 4 non-NaN values
        mask = n >= 4
        coef1 = n*(n+1)/((n-1)*(n-2)*(n-3))
        coef2 = 3*(n-1)**2/((n-2)*(n-3))
        res = coef1 * (m4 / (m2**2)) - coef2
        out[window-1:][mask[window-1:]] = res[mask]

        return out

    # 2. PyTorch backend
    x = torch.from_numpy(array).double().to(device)
    sw = x.unfold(0, window, 1)  # shape [T-window+1, F, window]
    m = torch.nanmean(sw, dim=2, keepdim=True)
    dev = sw - m
    m2 = torch.nanmean(dev.pow(2), dim=2)
    m4 = torch.nanmean(dev.pow(4), dim=2)
    n = torch.sum(~torch.isnan(sw), dim=2).double()

    coef1 = n*(n+1)/((n-1)*(n-2)*(n-3))
    coef2 = 3*(n-1).pow(2)/((n-2)*(n-3))
    kurt = coef1 * (m4 / m2.pow(2)) - coef2
    kurt = torch.where(n < 4, torch.full_like(kurt, float('nan')), kurt)

    pad = torch.full((window-1, F), float('nan'), dtype=kurt.dtype, device=device)
    out_t = torch.cat([pad, kurt], dim=0).cpu().numpy()
    return out_t


In [ ]:
import pandas as pd

arr = np.random.randn(200, 5)
np_k = rolling_kurt(arr, 10, backend="numpy")
pt_k = rolling_kurt(arr, 10, backend="torch")
pd_k = pd.DataFrame(arr).rolling(10).kurt().to_numpy()




False

In [56]:
pd_k

array([[            nan,             nan,             nan,
                    nan,             nan],
       [            nan,             nan,             nan,
                    nan,             nan],
       [            nan,             nan,             nan,
                    nan,             nan],
       [            nan,             nan,             nan,
                    nan,             nan],
       [            nan,             nan,             nan,
                    nan,             nan],
       [            nan,             nan,             nan,
                    nan,             nan],
       [            nan,             nan,             nan,
                    nan,             nan],
       [            nan,             nan,             nan,
                    nan,             nan],
       [            nan,             nan,             nan,
                    nan,             nan],
       [-4.62345737e-01, -1.22766572e+00, -6.17006130e-01,
        -2.22103031e-01

In [24]:
import ray
import pandas as pd

# 初始化 Ray
ray.init()

# 创建 DataFrame 列表
dfs = [pd.DataFrame({'a': [1, 2], 'b': [3, 4]}) for _ in range(3)]

# 放入对象存储
dfs_ref = ray.put(dfs)

@ray.remote
def process_data(data_ref):
    data = ray.get(data_ref)
    # 显式转换类型
    processed = [pd.DataFrame(df) for df in data]
    return processed

result = ray.get(process_data.remote(dfs_ref))
print(type(result[0]))  # 应该显示 <class 'pandas.core.frame.DataFrame'>

ModuleNotFoundError: No module named 'ray'

In [21]:
import re
def extract_value_from_string(input_string: str):
    """
    Extract values from a string in the format 'func(a,b)' or 'funcN'.
    Returns a tuple of extracted values, or None if no match is found.
    """
    match = re.match(r'(\w+)\((\d+),(\d+)\)', input_string)
    if match:
        print('check')
        return match.group(1), int(match.group(2)), int(match.group(3))
    match = re.match(r'(\w+)_(\d+)', input_string)
    if match:
        return match.group(1), int(match.group(2))
    return None


In [23]:
extract_value_from_string('aroonosc_6')

('aroonosc', 6)

In [6]:
from itertools import permutations
func_list = ['func1', 'func2', 'func3', 'func4']

In [14]:
func_tuple_list = [
			perm for i in range(1, min(4, len(func_list) + 1))
			for perm in permutations(func_list, i)
		]

In [15]:
func_tuple_list

[('func1',),
 ('func2',),
 ('func3',),
 ('func4',),
 ('func1', 'func2'),
 ('func1', 'func3'),
 ('func1', 'func4'),
 ('func2', 'func1'),
 ('func2', 'func3'),
 ('func2', 'func4'),
 ('func3', 'func1'),
 ('func3', 'func2'),
 ('func3', 'func4'),
 ('func4', 'func1'),
 ('func4', 'func2'),
 ('func4', 'func3'),
 ('func1', 'func2', 'func3'),
 ('func1', 'func2', 'func4'),
 ('func1', 'func3', 'func2'),
 ('func1', 'func3', 'func4'),
 ('func1', 'func4', 'func2'),
 ('func1', 'func4', 'func3'),
 ('func2', 'func1', 'func3'),
 ('func2', 'func1', 'func4'),
 ('func2', 'func3', 'func1'),
 ('func2', 'func3', 'func4'),
 ('func2', 'func4', 'func1'),
 ('func2', 'func4', 'func3'),
 ('func3', 'func1', 'func2'),
 ('func3', 'func1', 'func4'),
 ('func3', 'func2', 'func1'),
 ('func3', 'func2', 'func4'),
 ('func3', 'func4', 'func1'),
 ('func3', 'func4', 'func2'),
 ('func4', 'func1', 'func2'),
 ('func4', 'func1', 'func3'),
 ('func4', 'func2', 'func1'),
 ('func4', 'func2', 'func3'),
 ('func4', 'func3', 'func1'),
 ('fun

In [4]:
l = ''
if l:
    print("List is not empty")

In [3]:
str.join('_',
          ['30', 'b10', 'd1'])  # Output: '30_b10_d1'

'30_b10_d1'

In [2]:
'30_b10_d1'.split('_')[1:].join("_")

AttributeError: 'list' object has no attribute 'join'

In [47]:
a = torch.tensor([0, 1,2,3,4]).unsqueeze(0)
torch.nn.functional.pad(a, (2, 3), mode='reflect')

tensor([[2, 1, 0, 1, 2, 3, 4, 3, 2, 1]])